In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from src.dataset.coco import get_coco
import src.dataset.DatasetReader as DatasetReader
import matplotlib.pyplot as plt
import src.dataset.utils as utils
import time

In [ ]:
if torch.cuda.is_available():
    print("using cuda")
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("using mps")
    device = torch.device("mps")
else:
    print("using cpu")
    device = torch.device("cpu")

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

dataset = DatasetReader.COCODataset(annotation_file='../data/annotations/annotations/instances_train2017.json',
    image_dir= '../data/train2017/train2017',
    target_classes=[s.lower() for s in utils.GLOBAL_CLASSES],
    transform=transform)

batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=11, prefetch_factor=4, persistent_workers=True)

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 64)  
model = model.to(device)

print(len(dataset))
print(dataset[90001][1])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00002)

def train_model(model, dataloader, criterion, optimizer, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        for i, (images, labels) in enumerate(dataloader):

            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")
        asdf = time.time() - start_time
        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss / len(dataloader):.4f}, Time: {asdf:.4f}s")
        torch.save(model.state_dict(), f"model_{epoch}epoha_coco_train_batch_size_{batch_size}.pth")

In [ ]:
# run this to train the model 
# train_model(model, dataloader, criterion, optimizer)

# Evaluation
- treba da se stavi da koristi validacioni dataaset

In [ ]:
model.eval()

totalPost = 0
accuracyPost = 0


for i in range(0, len(dataset)):
    totalPost += 1
    image, label = dataset[i]
    image = image.unsqueeze(0).to(device)
    label = torch.tensor([label]).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    # print(f"Predicted: {predicted.item()}, Actual: {label.item()}")
    if(i % 1000 == 0):
        print(f"Slika: {i}")
    if predicted.item() == label.item():
        accuracyPost += 1
    

print(f"Accuracy: {accuracyPost/totalPost}")

In [ ]:
# ovo save-uje model
torch.save(model.state_dict(), "model_50epoch_coco_val_batch_size_384_acc_79.2.pth")

In [ ]:
model2 = models.mobilenet_v3_small(pretrained=True)
model2.classifier[3] = nn.Linear(model2.classifier[3].in_features, 64)  
model2 = model2.to(device)


model2.train()

for param in model2.features.parameters():

    param.requires_grad = False



for param in model2.classifier.parameters():

    param.requires_grad = True




In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.00002)

train_model(model2, dataloader, criterion, optimizer, num_epochs=50)